# 外皮情報LV1からLV2へのコンバート

## 1. 外皮簡易法＋用途別床面積
- 当該住戸の外皮の部位の面積等を用いずに簡易熱負荷計算を実施する場合には、部位の面積および長さを、「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）」２．エネルギー消費性能の算定方法　2.2　算定方法　第三章　 暖冷房負荷と外皮性能　	第二節　外皮性能　9.当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法　に示された標準住戸における部位の面積および長さ等より推定する。
- 屋根・天井、外壁、床の構成は、天井入力値である部位種別の熱貫流率と一致するよう、木造の標準的な構成における断熱材の厚さを0mm以上の範囲で調整する。木造の標準的な構成は、SimHeatでの熱負荷計算用の層構成決定に用いたQ値計算における、旧Ⅳ地域の平成11年基準の層構成とする。
- 土間床等の外周部の線熱貫流率については、簡易熱負荷計算での扱いに従う
- 開口部の日射熱取得率および外部日除けの取得日射熱補正係数は、それぞれ、季節別に、垂直面日射熱取得率および取得日射熱補正係数の入力値もしくは規定値を使用する。この場合、簡易熱負荷計算においては、開口部の入射角特性および外部日除けは考慮しない。
- その他は2.と同様に求める。

#### 《設計住戸の各部位の長さおよび面積の推定》
$$ \qquad
A_{i,d}= A_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (1)
\\
$$
$$ \qquad
L_{i,d}= L_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (2)
\\
$$

- 入力値：

  - 標準住戸の延床面積$[m ^ 2]$：$A_{floor,s}$
  - 設計住戸の延床面積$[m ^ 2]$：$A_{floor,d}$
  - 標準住戸の部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,s}$
  - 標準住戸の部位（土間床等の外周部）𝑖の長さ$[m]$：$L_{i,s}$

- 出力値：

  - 設計住戸の部位（一般部位又は開口部）𝑖の面積の推定値$[m ^ 2]$：$A_{i,d}$
  - 設計住戸の部位（土間床等の外周部）𝑖の長さの推定値$[m]$：$L_{i,d}$

- 屋根・天井壁体構成

| 材料 | 厚さ[m] | 熱伝導率[W/(m・K)] | 熱抵抗[（㎡K・W）] | 容積比熱[J/（L・K）] |
|-------------|-------------|-------------|-------------|
| Ro | - | - | 0.090 | 0 |
| 住宅用グラスウール断熱材10K相当 | <font color="red">0.2</font> | 0.05 | <font color="red">4.000</font> | 8 |
| せっこうボード | 0.0095 | 0.22 | 0.043 | 830 |
| Ri | - | - | 0.090 | 0 |

- 外壁体構成

| 材料 | 厚さ[m] | 熱伝導率[W/(m・K)] | 熱抵抗[（㎡K・W）] | 容積比熱[J/（L・K）] |
|-------------|-------------|-------------|-------------|
| Ro | - | - | 0.110 | 0 |
| 合板 | 0.012 | 0.16 | 0.075 | 720 |
| 住宅用グラスウール断熱材16K相当 | <font color="red">0.099</font> | 0.045 | <font color="red">2.200</font> | 13 |
| 密閉空気層 | - | - | 0.090 | 0 |
| せっこうボード | 0.0095 | 0.22 | 0.043 | 830 |
| Ri | - | - | 0.110 | 0 |

- 床壁体構成

| 材料 | 厚さ[m] | 熱伝導率[W/(m・K)] | 熱抵抗[（㎡K・W）] | 容積比熱[J/（L・K）] |
|-------------|-------------|-------------|-------------|
| Ro | - | - | 0.15 | - | - |
| 住宅用グラスウール断熱材16K相当 | <font color="red">0.099</font> | 0.045 | <font color="red">2.200</font> | 13 |
| 合板 | 0.012 | 0.16 | 0.075 | 720 |
| Ri | - | - | 0.15 | - | - |

In [1]:
#地域の区分 (1~8)
def Calc_Rg2(Rg1):
    Rg2 =  Rg1
    return Rg2

Rg1=6
print ("Rg2={0}".format(Calc_Rg2(Rg1)))

Rg2=6


In [4]:
#その他の居室の有無 (yes/no)
#非居室の有無 (yes/no)
def Calc_Flag2(Flag1):
    Flag2 =  Flag1
    return Flag2

Flag21=1
print ("Flag22={0}".format(Calc_Flag2(Flag21)))

Flag31=1
print ("Flag32={0}".format(Calc_Flag2(Flag31)))

Flag22=1
Flag32=1


In [2]:
#床面積の合計 (m2)
#主たる居室の床面積 (m2)
#その他の居室の床面積 (m2)
def Calc_A2(A1,Aa1,Aa2):
    A2 =  A1*Aa2/Aa1
    return A2

Aa1=120
Aa2=90
print ("Aa2={0}".format(Calc_A2(Aa1,Aa1,Aa2)))

A11=30
print ("A12={0}".format(Calc_A2(A11,Aa1,Aa2)))

A21=50
print ("A22={0}".format(Calc_A2(A21,Aa1,Aa2)))

Aa2=90.0
A12=22.5
A22=37.5


In [10]:
""" 部位別の面積の算出 """
def Calc_Ap2(Ap1):
    Ap2 =  Ap1
    return Ap2

Ap1=1
print ("Ap2={0}".format(Calc_Ap2(Ap1)))

Ap2=1


In [13]:
""" 部位別の長さの算出 """
def Calc_Lp2(Lp1):
    Lp2 =  Lp1
    return Lp2

Lp1=1
print ("Lp2={0}".format(Calc_Lp2(Lp1)))

Lp2=1


In [17]:
""" 部位別の熱貫流率の算出 """
def Calc_Up2(Up1):
    Up2 =  Up1
    return Up2

Up1=0.5
print ("Up2={0}".format(Calc_Up2(Up1)))

Up2=0.5


In [14]:
""" 部位別の線熱貫流率の算出 """
def Calc_Ψp2(Ψp1):
    Ψp2 =  Ψp1
    return Ψp2

Ψp1=1
print (Calc_Ψp2(Ψp1))

1


In [16]:
""" 部位別の日射熱取得率の算出 """
def Calc_ηp2(ηp1):
    ηp2 =  ηp1
    return ηp2

ηp1=0.8
print ("ηp2={0}".format(Calc_ηp2(ηp1)))

ηp2=0.8
